Follow Step 1 from the below link and get your Kaggle Token
https://medium.com/@yvettewu.dw/tutorial-kaggle-api-google-colaboratory-1a054a382de0
---
Upload best_model.pth to drive or runtime of colab
https://drive.google.com/file/d/1bVVZghsfU7q5lpSwGSNTSr36MIWCCId3/view?usp=sharing


In [0]:
from google.colab import files


In [0]:
!pip install -q kaggle

Upload the kaggle token downloaded.

In [0]:
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [0]:
!ls


kaggle.json  sample_data


In [0]:
!mkdir -p ~/.kaggle



In [0]:
!cp kaggle.json ~/.kaggle/



In [0]:
!kaggle datasets download -d xainano/handwrittenmathsymbols

 99% 339M/343M [00:01<00:00, 185MB/s]
100% 343M/343M [00:01<00:00, 183MB/s]


In [0]:
!unzip handwrittenmathsymbols.zip -d handwrittenmathsymbols


Streaming output truncated to the last 5000 lines.
  inflating: handwrittenmathsymbols/extracted_images/0/0_2414.jpg  
  inflating: handwrittenmathsymbols/extracted_images/0/0_24177.jpg  
  inflating: handwrittenmathsymbols/extracted_images/0/0_2420.jpg  
  inflating: handwrittenmathsymbols/extracted_images/0/0_24398.jpg  
  inflating: handwrittenmathsymbols/extracted_images/0/0_24469.jpg  
  inflating: handwrittenmathsymbols/extracted_images/0/0_2462.jpg  
  inflating: handwrittenmathsymbols/extracted_images/0/0_24690.jpg  
  inflating: handwrittenmathsymbols/extracted_images/0/0_24761.jpg  
  inflating: handwrittenmathsymbols/extracted_images/0/0_2490.jpg  
  inflating: handwrittenmathsymbols/extracted_images/0/0_24982.jpg  
  inflating: handwrittenmathsymbols/extracted_images/0/0_2501.jpg  
  inflating: handwrittenmathsymbols/extracted_images/0/0_25053.jpg  
  inflating: handwrittenmathsymbols/extracted_images/0/0_25274.jpg  
  inflating: handwrittenmathsymbols/extracted_images/0/0_

In [0]:
get_ipython().system_raw("unrar x handwrittenmathsymbols/data.rar")


In [0]:
import numpy as np
import pandas as pd

In [0]:
!pip install split-folders

In [0]:
import os
import numpy as np
from tqdm import tqdm
from fastai import *
import torch
from fastai.vision import *
%matplotlib inline
from fastai.callbacks import *
import cv2
import pandas as pd
import split_folders

In [0]:
image_load_size = 64
bs = 24

In [0]:
 def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True    

SEED = 999
seed_everything(SEED)

In [0]:
split_folders.ratio('extracted_images', output="final_output_images", seed=SEED, ratio=(.8, .2))

Copying files: 375974 files [00:40, 9175.82 files/s]


In [0]:
tfms = get_transforms(do_flip=True, flip_vert=False, max_lighting=0.1, max_zoom=1.05,
                      max_warp=0.,
                      xtra_tfms=[rand_crop(), rand_zoom(1, 1.5),
                                 symmetric_warp(magnitude=(-0.2, 0.2))])

In [0]:
data = (ImageList.from_folder(path='final_output_images')
        .split_by_folder(train='train',valid='val')
        .label_from_folder()
        .transform(tfms,size = image_load_size,resize_method=ResizeMethod.SQUISH)
        .databunch(path='.',bs=bs)    
        .normalize(imagenet_stats)
)

In [0]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1., gamma=2.):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets, **kwargs):
        CE_loss = nn.CrossEntropyLoss(reduction='none')(inputs, targets)
        pt = torch.exp(-CE_loss)
        F_loss = self.alpha * ((1-pt)**self.gamma) * CE_loss
        return F_loss.mean()

In [0]:
model = cnn_learner(data,models.densenet161, metrics = [accuracy,error_rate],callback_fns=[partial(SaveModelCallback, monitor='accuracy', name='best_model')])
model.loss_func = FocalLoss()
model.summary()

Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /root/.cache/torch/checkpoints/densenet161-8d451a50.pth


Sequential
Layer (type)         Output Shape         Param #    Trainable 
Conv2d               [96, 32, 32]         14,112     False     
______________________________________________________________________
BatchNorm2d          [96, 32, 32]         192        True      
______________________________________________________________________
ReLU                 [96, 32, 32]         0          False     
______________________________________________________________________
MaxPool2d            [96, 16, 16]         0          False     
______________________________________________________________________
BatchNorm2d          [96, 16, 16]         192        True      
______________________________________________________________________
ReLU                 [96, 16, 16]         0          False     
______________________________________________________________________
Conv2d               [192, 16, 16]        18,432     False     
___________________________________________________

Give the path of best_model.pth

In [0]:
model.load('/content/drive/My Drive/best_model')


Learner(data=ImageDataBunch;

Train: LabelList (300746 items)
x: ImageList
Image (3, 64, 64),Image (3, 64, 64),Image (3, 64, 64),Image (3, 64, 64),Image (3, 64, 64)
y: CategoryList
lambda,lambda,lambda,lambda,lambda
Path: final_output_images;

Valid: LabelList (75228 items)
x: ImageList
Image (3, 64, 64),Image (3, 64, 64),Image (3, 64, 64),Image (3, 64, 64),Image (3, 64, 64)
y: CategoryList
lambda,lambda,lambda,lambda,lambda
Path: final_output_images;

Test: None, model=Sequential(
  (0): Sequential(
    (0): Sequential(
      (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu0): ReLU(inplace=True)
      (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (denseblock1): _DenseBlock(
        (denselayer1): _DenseLayer(
          (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats

In [0]:
print(os.listdir('final_output_images/val/X'))


['exp95701.jpg', 'exp94236.jpg', 'exp8887.jpg', 'x_102409.jpg', 'X_79267.jpg', 'exp52052.jpg', 'exp88733.jpg', 'exp24426.jpg', 'exp36455.jpg', 'x_96560.jpg', 'x_84266.jpg', 'x_6832.jpg', 'exp52686.jpg', 'exp20425.jpg', 'exp70705.jpg', 'x_101244.jpg', 'exp9506.jpg', 'exp52017.jpg', 'exp12989.jpg', 'exp3081.jpg', 'exp71782.jpg', 'exp3262.jpg', 'exp9612.jpg', 'x_11218.jpg', 'X_59850.jpg', 'exp6144.jpg', 'exp48231.jpg', 'exp91820.jpg', 'exp59150.jpg', 'exp89871.jpg', 'exp69670.jpg', 'x_67645.jpg', 'x_77587.jpg', 'X_48699.jpg', 'exp59757.jpg', 'exp9292.jpg', 'x_24473.jpg', 'exp19196.jpg', 'exp35942.jpg', 'exp86109.jpg', 'x_99145.jpg', 'exp61780.jpg', 'exp89914.jpg', 'x_33657.jpg', 'x_65256.jpg', 'exp34515.jpg', 'exp71491.jpg', 'exp90905.jpg', 'exp24100.jpg', 'exp27760.jpg', 'exp43466.jpg', 'x_113373.jpg', 'exp63826.jpg', 'exp38551.jpg', 'exp6001.jpg', 'exp56686.jpg', 'exp72248.jpg', 'exp51560.jpg', 'exp41358.jpg', 'exp8548.jpg', 'exp20943.jpg', 'x_14616.jpg', 'exp94353.jpg', 'exp95485.jpg',

In [0]:
pred = model.predict(open_image('roi1_white.png'))
print(pred[0])


X
